In [33]:
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt

In [34]:

# cargamos el video
camara = cv2.VideoCapture("vga.mpg")

In [35]:

# Inicializamos el primer frame a vacio 
# este nos permitira obtener el fondo - segunda captura 
fondo = None
# recorrido de los frames del video
while True:
    # obtenemos le frame
    (grabbed, frame) = camara.read()
    
    # si llegamos al final del video el bucle se rompe
    if not grabbed:
        break
    
    # convertios a escala de grises
    gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # aplicamos suavizado para eliminar ruido
    gris = cv2.GaussianBlur(gris, (21, 21), 0)
    
    # Si todavia no hemos obtenido el fondo, lo obtenemos
    # sera el primer frame que obtengamos
    if fondo is None:
        fondo = gris
        continue
    
    # calculo de la diferencia entre el fondo y el frame actual
    resta = cv2.absdiff(fondo, gris)
    
    # aplicamos el umbral 
    umbral = cv2.threshold(resta, 25, 255, cv2.THRESH_BINARY)[1]
    
    # dilatamos el umbral para tapar agujeros
    umbral = cv2.dilate(umbral, None, iterations=2)
    
    # copiamos el umbral para detectar los contornos
    contornosimg = umbral.copy()
    
    # buscamos contorno en la imagen
    contornos, hierarchy = cv2.findContours(contornosimg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # recorremos todos los contornos encontrados
    for c in contornos:
        # eliminamos los contornos mas pequeños
        if cv2.contourArea(c) < 500:
            continue
        # Obtenemos el bounds del contorno, el rectangulo mayor que engloba al contrario 
        (x, y, w, h) = cv2.boundingRect(c)
        
        # dibujamos el rectangulo del bounds 
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
    
    # Mostramos las imágenes de la cámara, el umbral y la resta
    cv2.imshow("Camara", frame)
    cv2.imshow("Umbral", umbral)
    cv2.imshow("Resta", resta)
    cv2.imshow("Contorno", contornosimg)
    
    # capturamos una tecla para salir
    key = cv2.waitKey(1) & 0xFF
    # tiempo de espera para que se vea bien
    time.sleep(0.015)
    if key == ord("s"):
        break

# liberamos la camara y cerramos todas las ventanas

camara.release()
cv2.destroyAllWindows()
    
    
